## Trabajo Práctico NLP: Detección de Tópicos y Clasificación

**Mariela Iaccarino**

Certificación Experta en NLP - ITBA

#### Este código está diseñado para procesar tópicos diariamente, almacenarlos en una base de datos, comparar tópicos entre días y decidir si un tópico es nuevo o si debe mergearse con uno existente.

El flujo será el siguiente:

1) Entrenar el modelo usando solo los datos del primer día.

2) Guardar los tópicos generados en la base de datos.

3) Procesar los datos de los días siguientes, comparando los nuevos tópicos con los existentes en la base de datos.

4) Decidir si un tópico es nuevo o si debe mergearse con uno existente.

5) Generar nuevos tópicos si no hay coincidencias relevantes.

6) Generar Inferencia y Clasificación de Nuevos Documentos

## 1. Configuración y Carga de Datos

1.1 Importación de librerías necesarias:


In [ ]:
!pip install datasets umap-learn chromadb hdbscan sentence_transformers BERTopic opensearch-py matplotlib

In [ ]:
!pip install opensearch-py==2.3.0

In [ ]:
from datasets import load_dataset
import pandas as pd
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from datetime import datetime
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
from opensearch_data_model import Topic, TopicKeyword, os_client
from dateutil.parser import parse
from utils import SPANISH_STOPWORDS
import numpy as np
from datetime import timedelta
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)

/home/RECONQUISTA/a118672/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
Topic.init()

2. Configuración del Modelo de Embeddings y Otros Modelos:

Utilizaremos BERTopic para la detección de tópicos. Aplicaremos UMAP para la reducción de dimensionalidad y HDBSCAN para el clustering de los embeddings.

In [ ]:
# Configuración del modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Configuración de UMAP para reducción de dimensionalidad
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Configuración de HDBSCAN para clustering
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)




Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
# Configuración de pipelines para NER y análisis de sentimiento
ner_pipeline = pipeline('ner', model='dbmdz/bert-large-cased-finetuned-conll03-english')
sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Función para extracción de entidades
def extract_entities(text):
    entities = ner_pipeline(text)
    return [entity['word'] for entity in entities]

# Función para análisis de sentimiento
def analyze_sentiment(text):
    sentiment = sentiment_pipeline(text)
    return sentiment[0]


3. Funciones de Utilidad

In [ ]:

# Función para extracción de entidades
def extract_entities(text):
    entities = ner_pipeline(text)
    return [entity['word'] for entity in entities]

# Función para análisis de sentimiento
def analyze_sentiment(text):
    sentiment = sentiment_pipeline(text)
    return sentiment[0]


In [ ]:
def calculate_similarity_threshold(topic_id, topic_model, probs):
    topic_probs = [probs[i] for i, topic in enumerate(topic_model.topics_) if topic == topic_id]
    mean_prob = np.mean(topic_probs)
    std_dev_prob = np.std(topic_probs)
    threshold = mean_prob + 1.5 * std_dev_prob
    return threshold

In [ ]:
def extract_entities(text):
    entities = ner_pipeline(text)
    return entities

def analyze_sentiment(text):
    sentiment = sentiment_pipeline(text)
    return sentiment


In [ ]:
def get_topic_name(keywords):
    # Verificar si 'keywords' es una lista
    if isinstance(keywords, list) and len(keywords) > 0:
        # Verificar la estructura del primer elemento de 'keywords'
        first_element = keywords[0]

        # Si el primer elemento es una tupla o lista, intentamos extraer el primer elemento de cada tupla/lista
        if isinstance(first_element, (tuple, list)):
            try:
                # Intentamos descomponer asumiendo que hay al menos dos elementos en la tupla/lista
                return ', '.join([k[0] for k in keywords[:4]])
            except (IndexError, ValueError):
                # Si hay un problema al descomponer, devolvemos las keywords como están
                return ', '.join([str(k) for k in keywords[:4]])
        else:
            # Si el primer elemento no es una tupla/lista, asumimos que es una lista de strings
            return ', '.join([str(k) for k in keywords[:4]])
    else:
        return "Tópico Desconocido"




In [ ]:
def topic_threshold(topic_id, topic_model, probs):
    try:
        docs_per_topics =  [i for i, x in enumerate(topic_model.topics_) if x== topic_id    ]
        return np.array(  [probs[doc_idx] for doc_idx in docs_per_topics  ]).mean()
    except:
        return 0

In [ ]:
def delete_index_opensearch(index_name: str) -> bool:

    try:
        # Consulta para eliminar todos los documentos
        delete_query = {
                        "query": {
                        "match_all": {}
                        }
        }

        # Ejecutar la operación de borrado por consulta
        response = os_client.delete_by_query(index=index_name, body=delete_query)

        return True

    except Exception as e:
        print(f"Ha ocurrido un error: {e}")
        return

3. Generación del Datase

In [ ]:
# Cargar datasets y agregar el campo de fecha
def add_fecha_field(dataset, fecha):
    df = pd.DataFrame(dataset['train'])
    df['date'] = fecha
    return df

datasets = [
    ("jganzabalseenka/news_2024-07-01_24hs", '2024-07-01'),
    ("jganzabalseenka/news_2024-07-12_24hs", '2024-07-12'),
    ("jganzabalseenka/news_2024-07-14_24hs", '2024-07-14'),
    ("jganzabalseenka/news_2024-07-16_24hs", '2024-07-16'),
    ("jganzabalseenka/news_2024-07-19_24hs", '2024-07-19')
]

df_list = [add_fecha_field(load_dataset(ds[0]), ds[1]) for ds in datasets]
df = pd.concat(df_list, ignore_index=True)


## 2. Procesamiento y Detección de Tópicos Diarios

Cada día, procesaremos los textos y determinaremos los tópicos. Luego, compararemos estos tópicos con los días anteriores para decidir si se deben mergear con tópicos existentes o si deben crearse como nuevos.

### Paso 1: Procesamiento de Datos del Primer Día

In [ ]:
def process_first_day(df):
    # Definir el vectorizador usando todas las entidades y keywords únicas del dataset
    entities = set(sum(list([list(e) for e in df['entities_transformers'].values]), []))
    keywords = set(sum(list([list(e) for e in df['keywords'].values]), []))
    all_tokens = list(entities.union(keywords))

    # Configurar el vectorizador
    tf_vectorizer = CountVectorizer(
        ngram_range=(1, 3),
        stop_words=SPANISH_STOPWORDS,
        lowercase=False,
        vocabulary=all_tokens,
    )

    # Configurar UMAP y HDBSCAN para el modelo BERTopic
    umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
    hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
# Configuración de HDBSCAN para clustering

    # Configurar el modelo BERTopic
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=tf_vectorizer,
        ctfidf_model=ClassTfidfTransformer(),
        representation_model=KeyBERTInspired(),
        language='spanish'
    )

    # Entrenar el modelo con los textos del primer día
    topics, probs = topic_model.fit_transform(df['text'].tolist())

    # Generar embeddings de los textos usando el modelo de embeddings

    texts = df['text'].tolist()
    embedings = topic_model.embedding_model.embed(texts)
    sim_matrix = cosine_similarity( topic_model.topic_embeddings_, embedings)

    for topic in topic_model.get_topics().keys():
        if topic > -1:
            print(topic)
            keywords = topic_model.topic_representations_[topic]
            topic_keywords = [TopicKeyword(name=k, score=s) for k, s in keywords]
            threshold= topic_threshold(topic,topic_model,probs)
            from_date = parse(df['date'].min())
            to_date = from_date + timedelta(days=1)

            best_doc_index = sim_matrix[topic + 1].argmax()

            best_doc = df.iloc[best_doc_index].text

            topic_doc = Topic(
                vector = list(topic_model.topic_embeddings_[topic + 1]),
                similarity_threshold=threshold,
                created_at = datetime.now(),
                to_date = to_date,
                from_date = from_date,
                index = topic,
                keywords = topic_keywords,
                name = get_topic_name(keywords),
                best_doc = best_doc
            )

            print(topic_doc.save())

In [ ]:
delete_index_opensearch("topic")

In [ ]:
# Procesar primer día
first_day_df = df[df['date'] == df['date'].min()]
process_first_day(first_day_df)

/home/RECONQUISTA/a118672/.local/lib/python3.8/site-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df) + 1)


### Paso 2: Merge de Tópicos entre Días

Para evitar la proliferación de tópicos redundantes, implementaremos un criterio de agrupación de tópicos que se aplica dentro de un mismo día y entre días diferentes. Usaremos cosine similarity para realizar la comparación:

In [ ]:
def merge_topics(new_topic_embedding, os_client, new_topic_threshold):
    existing_topics = Topic.search().execute()

    for existing_topic in existing_topics:
        similarity = cosine_similarity([new_topic_embedding], [existing_topic.vector])[0][0]

        if similarity > existing_topic.similarity_threshold:
            return existing_topic

    return None


### Paso 3: Procesamiento de Días Siguientes

Ahora procesamos los días siguientes, comparando los tópicos con los ya existentes y decidiendo si mergear o crear un nuevo tópico.

In [ ]:
import urllib3
import numpy as np
from datetime import datetime
from dateutil.parser import parse

# Suprimir las advertencias SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def process_subsequent_days(df, date):
    # Generar embeddings de los textos usando el modelo de embeddings
    df['embeddings'] = df['text'].apply(lambda x: embedding_model.encode(x))

    for i, row in df.iterrows():
        # Mostrar el número de tópicos antes de procesar
        initial_topic_count = Topic.search().count()
        print(f"Número inicial de tópicos en OpenSearch: {initial_topic_count}")

        # Verificar si el tópico ya existe
        existing_topic = merge_topics(row['embeddings'], os_client, None)

        if existing_topic:
            print(f"Mergeando tópico existente: {existing_topic.index}")

            # Mergear con el tópico existente
            existing_topic_vector = np.array(existing_topic.vector)
            new_embedding_vector = np.array(row['embeddings'])
            averaged_vector = (existing_topic_vector + new_embedding_vector) / 2

            # Convertir a lista antes de guardar
            existing_topic.vector = averaged_vector.tolist()

            # Actualizar las keywords en el tópico existente
            existing_keywords = [kw.name for kw in existing_topic.keywords]
            combined_keywords = list(set(existing_keywords + row['keywords']))
            existing_topic.keywords = [TopicKeyword(name=k, score=1.0) for k in combined_keywords]

            try:
                result = existing_topic.save()
                print(f"Tópico existente guardado con éxito: {result}")
            except ValueError as e:
                print(f"Error al guardar el tópico existente: {str(e)}")
                continue
        else:
            print(f"Creando nuevo tópico para el documento en índice {i}")

            # Crear un nuevo tópico si no se encuentra uno existente
            topic_name = get_topic_name(row['keywords'])
            new_topic_doc = Topic(
                vector=row['embeddings'].tolist(),
                similarity_threshold=0.75,
                created_at=datetime.now(),
                to_date=datetime.now(),
                from_date=datetime.strptime(date, '%Y-%m-%d'),
                index=i,
                name=topic_name,
                best_doc=row['text'],
                keywords=[TopicKeyword(name=k, score=1.0) for k in row['keywords']],
            )

            try:
                result = new_topic_doc.save()
                print(f"Nuevo tópico guardado con éxito: {result}")
            except ValueError as e:
                print(f"Error al guardar el nuevo tópico: {str(e)}")
                continue

        # Mostrar el número de tópicos después de procesar
        updated_topic_count = Topic.search().count()
        print(f"Número actualizado de tópicos en OpenSearch: {updated_topic_count}")

# Simular procesamiento de días siguientes
for date in df['date'].unique()[1:]:
    print(f"\nProcesando tópicos para la fecha: {date}")
    daily_df = df[df['date'] == date]
    process_subsequent_days(daily_df, date)

# Mostrar el número total de tópicos al final
final_topic_count = Topic.search().count()
print(f"Número total final de tópicos en OpenSearch: {final_topic_count}")



Procesando tópicos para la fecha: 2024-07-12
Número inicial de tópicos en OpenSearch: 181
Mergeando tópico existente: 15
Tópico existente guardado con éxito: updated
Número actualizado de tópicos en OpenSearch: 181
Número inicial de tópicos en OpenSearch: 181
Mergeando tópico existente: 15
Tópico existente guardado con éxito: updated
Número actualizado de tópicos en OpenSearch: 181
Número inicial de tópicos en OpenSearch: 181
Mergeando tópico existente: 15
Tópico existente guardado con éxito: updated
Número actualizado de tópicos en OpenSearch: 181
Número inicial de tópicos en OpenSearch: 181
Mergeando tópico existente: 15
Tópico existente guardado con éxito: updated
Número actualizado de tópicos en OpenSearch: 181
Número inicial de tópicos en OpenSearch: 181
Mergeando tópico existente: 15
Tópico existente guardado con éxito: updated
Número actualizado de tópicos en OpenSearch: 181
Número inicial de tópicos en OpenSearch: 181
Mergeando tópico existente: 15
Tópico existente guardado co

In [ ]:
Topic.search().count()

181

## 3. Clasificación de Nuevos Documentos

In [ ]:
def classify_text(title, text):
    # Combinar título y texto
    combined_text = title + " " + text

    # Generar embeddings del texto combinado
    new_embed = embedding_model.encode([combined_text])  # Cambiado de embed() a encode()

    # Construir la consulta KNN para OpenSearch
    query = {
        "size": 5,
        "query": {
            "knn": {
                "vector": {
                    "vector": new_embed[0].tolist(),
                    "k": 1000
                }
            }
        }
    }

    # Ejecutar la búsqueda en OpenSearch
    response = os_client.search(index='topic', body=query)

    # Verificar si se encontraron resultados
    if response['hits']['hits']:
        # Obtener la información del tópico más cercano
        topic_id = response['hits']['hits'][0]['_id']
        keywords = response['hits']['hits'][0]['_source']['keywords']

        # Extraer entidades y análisis de sentimiento
        entities = extract_entities(combined_text)
        sentiment = analyze_sentiment(combined_text)

        # Devolver los resultados
        return topic_id, keywords, entities, sentiment

    # Si no se encuentra ningún resultado, devolver None
    return None, None, None, None


# Ejemplo de clasificación
new_title = "Argentina Campeon"
new_text = "Termino la copa America y Argentina salio campeon"
topic_id, keywords, entities, sentiment = classify_text(new_title, new_text)
print(f"Tópico: {topic_id}, Keywords: {keywords}, Entidades: {entities}, Sentimiento: {sentiment}")


Tópico: 106_Bolivia_Panamá,_Estados_Unidos,_Uruguay,_Panamá, Keywords: [{'name': 'Bolivia Panamá', 'score': 0.6121788024902344}, {'name': 'Estados Unidos', 'score': 0.5404210686683655}, {'name': 'Uruguay', 'score': 0.5317678451538086}, {'name': 'Panamá', 'score': 0.5012048482894897}, {'name': 'Estados', 'score': 0.47730502486228943}, {'name': 'Bolivia', 'score': 0.45392051339149475}, {'name': 'José Córdoba', 'score': 0.3842011094093323}, {'name': 'Roberto Fernández', 'score': 0.34443116188049316}, {'name': 'Matías Viña', 'score': 0.32149630784988403}, {'name': 'José Sagredo', 'score': 0.3189774453639984}], Entidades: ['Argentina', 'Camp', 'America', 'Argentina'], Sentimiento: {'label': 'POSITIVE', 'score': 0.7363261580467224}


In [ ]:
def classify_text(title, text):
    # Combinar título y texto
    combined_text = title + " " + text

    # Generar embeddings del texto combinado
    new_embed = embedding_model.encode([combined_text])  # Asegúrate de usar encode() si es SentenceTransformer

    # Construir la consulta KNN para OpenSearch
    query = {
        "size": 5,
        "query": {
            "knn": {
                "vector": {
                    "vector": new_embed[0].tolist(),
                    "k": 1000
                }
            }
        }
    }

    # Ejecutar la búsqueda en OpenSearch
    response = os_client.search(index='topic', body=query)

    # Verificar si se encontraron resultados
    if response['hits']['hits']:
        # Obtener la información del tópico más cercano
        topic_id = response['hits']['hits'][0]['_id']
        keywords = response['hits']['hits'][0]['_source']['keywords']
        name = response['hits']['hits'][0]['_source']['name']  # Obtener el nombre del tópico
        best_doc = response['hits']['hits'][0]['_source']['best_doc']  # Obtener el mejor documento

        # Extraer entidades y análisis de sentimiento
        entities = extract_entities(combined_text)
        sentiment = analyze_sentiment(combined_text)

        # Devolver los resultados
        return topic_id, name, keywords, best_doc, entities, sentiment

    # Si no se encuentra ningún resultado, devolver None
    return None, None, None, None, None, None


# Ejemplo de clasificación
new_title = "Argentina Campeon"
new_text = "Termino la copa America y Argentina salio campeon"
topic_id, name, keywords, best_doc, entities, sentiment = classify_text(new_title, new_text)
print(f"Tópico: {topic_id}, Nombre: {name}, Keywords: {keywords}, Best Doc: {best_doc}, Entidades: {entities}, Sentimiento: {sentiment}")


Tópico: 106_Bolivia_Panamá,_Estados_Unidos,_Uruguay,_Panamá, Nombre: Bolivia Panamá, Estados Unidos, Uruguay, Panamá, Keywords: [{'name': 'Bolivia Panamá', 'score': 0.6121788024902344}, {'name': 'Estados Unidos', 'score': 0.5404210686683655}, {'name': 'Uruguay', 'score': 0.5317678451538086}, {'name': 'Panamá', 'score': 0.5012048482894897}, {'name': 'Estados', 'score': 0.47730502486228943}, {'name': 'Bolivia', 'score': 0.45392051339149475}, {'name': 'José Córdoba', 'score': 0.3842011094093323}, {'name': 'Roberto Fernández', 'score': 0.34443116188049316}, {'name': 'Matías Viña', 'score': 0.32149630784988403}, {'name': 'José Sagredo', 'score': 0.3189774453639984}], Best Doc: El seleccionado centroamericano va por la victoria que le de la chance de avanzar en la Copa América.

Bolivia y Panamá se enfrentarán este lunes a las 22:00 en el Inter&Co Stadium de Orlando, por la tercera jornada del Grupo C de la Copa América. El partido será transmitido por D Sports, con Edina Alves como árbitro 

Resumen
Este código integra todas las funcionalidades:

* Procesamiento diario de tópicos: Los tópicos se detectan y procesan diariamente, almacenándolos en la base de datos.

* Comparación de tópicos entre días: Los tópicos se comparan con los existentes para determinar si deben mergearse o si deben crearse como nuevos.

* Clasificación de nuevos documentos: Cuando se recibe un nuevo documento, se compara con los tópicos existentes y se clasifica se extraen entidades y keywords, y se realiza un análisis de sentimiento.

* Almacenamiento en OpenSearch: Los tópicos y sus embeddings se almacenan en OpenSearch, permitiendo búsquedas eficientes y comparaciones entre días.